<a href="https://colab.research.google.com/github/GoodnessOno/MEDICAL-AI-MODEL/blob/main/Medical_Diagnosis_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# 2. Unzip your file (update path if needed)
import zipfile
import os

zip_path = '/content/drive/MyDrive/hospital-archive.zip'  # Update this
extract_path = '/content/disease-dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete.")


Extraction complete.


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/hospital-archive/dataset.csv')

# Fill NaNs with empty strings
df.fillna('', inplace=True)

# Step 1: Convert symptoms into a single list per row
symptom_cols = [col for col in df.columns if col.startswith("Symptom_")]
df['all_symptoms'] = df[symptom_cols].apply(lambda row: [s for s in row if s != ''], axis=1)

# Step 2: Build the complete set of unique symptoms
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
symptom_matrix = mlb.fit_transform(df['all_symptoms'])

# Step 3: Create the final feature dataframe
X = pd.DataFrame(symptom_matrix, columns=mlb.classes_)

# Step 4: Encode target labels (Disease)
le = LabelEncoder()
y = le.fit_transform(df['Disease'])

# Step 5: Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 7: Predict and evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))


Accuracy: 1.0
                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00        18
                                   AIDS       1.00      1.00      1.00        30
                                   Acne       1.00      1.00      1.00        24
                    Alcoholic hepatitis       1.00      1.00      1.00        25
                                Allergy       1.00      1.00      1.00        24
                              Arthritis       1.00      1.00      1.00        23
                       Bronchial Asthma       1.00      1.00      1.00        33
                   Cervical spondylosis       1.00      1.00      1.00        23
                            Chicken pox       1.00      1.00      1.00        21
                    Chronic cholestasis       1.00      1.00      1.00        15
                            Common Cold       1.00      1.00      1.00        23
             

In [5]:
import joblib

joblib.dump(model, 'medical_diagnosis_model.pkl')
joblib.dump(le, 'disease_label_encoder.pkl')
joblib.dump(mlb, 'symptom_binarizer.pkl')


['symptom_binarizer.pkl']